In [1]:
#Verificar versión de gpu Nvidia
!nvidia-smi

Thu Nov 19 01:22:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |    590MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [40]:
#Importar librerías a utilizar
import cudf, cuml, cuxfilter
from pandas.io.json import json_normalize 
import pandas as pd

In [48]:
#Fase de extracción y lectura con Rapids
import time

start = time.time()

#extracción y lectura con Pandas
#pd_Pandas = pd.read_csv('df_bigdata.csv')

#extraccion y lectura con cudf
gdf = cudf.read_csv('df_bigdata.csv')

end = time.time()
caculatedCUDF = end - start

print("Cálculo de extracción y lectura con cudf gpu-Rapdis= {}".format(caculatedCUDF))

Cálculo de extracción y lectura con cudf gpu-Rapdis= 0.9464094638824463


In [5]:
print('Descripción de la extracción y lectura del archivo:')
gdf

Descripción de la extracción y lectura del archivo:


,u,n,o
0,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
1,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
2,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
3,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
4,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
...,...,...,...
399995,/edx/var/log/tracking/tracking.log-20171220-15...,<NA>,<NA>
399996,/edx/var/log/tracking/tracking.log-20171220-15...,<NA>,<NA>
399997,/edx/var/log/tracking/tracking.log-20171220-15...,<NA>,<NA>
399998,/edx/var/log/tracking/tracking.log-20171220-15...,<NA>,<NA>


In [51]:
import json

cudf_Pandas = pd_Pandas

#crear formato json para despues crear un dataframe nuevo
start = time.time()
lista = list(cudf_Pandas['u'])

a = []
for i in range(0,len(lista)):
    d = lista[i].split('.gz:')
    a.append(json.loads((d[1])))
    
#Crear DataFrame con Pandas
pdPandas = pd.DataFrame(a)

# Convierto todo el dataframe a str
nuevo_pdPandas = pdPandas.applymap(str)

#solo conservo los datos de la columna username
#mayor a 0
snNull = nuevo_pdPandas[nuevo_pdPandas.username.map(len) > 0]

#elimino username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#convertir de pandas DataFrame
#aún cudf DataFrame
gdf_new = cudf.DataFrame.from_pandas(snCampus)

#Eliminar columnas que no tiene mucha relevancia
gdf_newData = gdf_new.drop(['accept_language','name', 'agent',
             'page', 'session', 'event', 'event_type'], axis=1)

#eliminar filas con los datos perdidos
gdf_newData = gdf_newData.dropna(axis=0, how='any')

#Dividir columna context estaba en json
newContext = gdf_newData['context'].to_arrow().to_pylist()
b=[]
for i in range(0,len(newContext)):
    d = newContext[i].split(',')
    b.append(d[1])
    
gdfContext = cudf.DataFrame()
gdfContext['General'] = b
gdfContext = gdfContext['General'].str.split()
gdfContext = gdfContext.drop(0, axis=1)

#Unir DataFrame context al general
newData = gdf_newData.join(gdfContext, how='left')
newData = newData.drop(['context'], axis=1)

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

#dividir columna time
vtime = newData['time']
vtime.str.normalize_spaces()
new_time = vtime.str.split(pat = "-")

day = new_time[2]
daytime = day.str.split(pat = 'T')
daytime.drop(1, axis=1)
daytime.columns = ['day', 'hour']

timenew = new_time.join(daytime, how='left')
timenew.columns = ['year','month','d','day','hour']
timenew = timenew.drop(['d'], axis=1)


#Unir todo al dataframe time al general
newData = newData.join(timenew, how='left')
newDataMejorada = newData.drop(['time'], axis=1)

#eliminar filas con datos perdidos
newDataMejorada.dropna(axis=0, how='any')

#resetear index
newDataMejorada = newDataMejorada.reset_index()
newDataMejorada = newDataMejorada.drop(['index'], axis=1)


end = time.time()
calculo_Transform= end - start

print('Cálculo de transformación con cudf gpu-Rapdis:', calculo_Transform)

/opt/conda-environments/rapids-stable/lib/python3.8/site-packages/cudf/core/column/string.py:1993: UserWarning: `expand` parameter defatults to True.
  warnings.warn("`expand` parameter defatults to True.")


Cálculo de transformación con cudf gpu-Rapdis: 15.70364761352539


In [11]:
#Exploración de todo el Dataset con cudf
print('Nombre de columnas:',newDataMejorada.columns)
print('Dimension DataFrame:',newDataMejorada.shape)
print('Tipos de datos de cada columna', newDataMejorada.dtypes)
print('Información DATA general con Rapids:')
newDataMejorada.head(2)

Nombre de columnas: Index([    'username', 'event_source',         'host',      'referer',
                 'ip',              1,         'year',        'month',
                'day',         'hour'],
      dtype='object')
Dimension DataFrame: (393734, 10)
Tipos de datos de cada columna username        object
event_source    object
host            object
referer         object
ip              object
1               object
year            object
month           object
day             object
hour            object
dtype: object
Información DATA general con Rapids:


,username,event_source,host,referer,ip,1,year,month,day,hour
0,Amada1012,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,190.57.140.77,9844,2017,12,19,04:23:10.318087+00:00
1,Amada1012,browser,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,190.57.140.77,9844,2017,12,19,04:23:11.007723+00:00


In [ ]:
###########Carga con Rapids#############

In [65]:
#Cargar la data en un nuevo csv
import time

start = time.time()

da_data = 'data_mejoraRapids_grande.csv'
newDataMejorada.to_csv(da_data)

end = time.time()
calculated = end - start

print("Cálculo carga con cudf gpu-Rapdis = {}".format(calculated))

Cálculo carga con cudf gpu-Rapdis = 1.032219648361206


In [20]:
#Extracción y lestura del nuevo archivo
#Transformado
nueva_data = cudf.read_csv('data_mejoraRapids_grande.csv')
nueva_data = nueva_data.drop(['Unnamed: 0'], axis=1)

#Información de la nueva data
print('Tipos de datos de cada columna', newData.dtypes)
print('Información Data')
nueva_data.head(2)

Tipos de datos de cada columna username        object
event_source    object
time            object
host            object
referer         object
ip              object
1               object
year            object
month           object
day             object
hour            object
dtype: object
Información Data


,username,event_source,host,referer,ip,1,year,month,day,hour
0,Amada1012,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,190.57.140.77,9844,2017,12,19,04:23:10.318087+00:00
1,Amada1012,browser,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,190.57.140.77,9844,2017,12,19,04:23:11.007723+00:00


In [ ]:
####Proceso Machine Learning##########

In [21]:
#Fase dividir Dataset
from cuml.preprocessing.model_selection import train_test_split

#Asignar dataset general a nuevas variables
gdf_dataX = nueva_data
gdf_dataY = nueva_data

#Dividir dataset en train y test
#datatrain 80% y datatest 20%
X_train, X_test, y_train, y_test = train_test_split(gdf_dataX, gdf_dataY,
                                                    train_size=0.8)
#información del dataset dividido
print(f'Dataset Original: {gdf_dataX.shape[0], gdf_dataY.shape[0]} elementos')
print(f'Data X_train: {X_train.shape[0]} elements')
print(f'Data X_test: {X_test.shape[0]} elements')
print(f'Data y_train: {y_train.shape[0]} elements')
print(f'Data y_test: {y_test.shape[0]} elements')

Dataset Original: (393734, 393734) elementos
Data X_train: 314987 elements
Data X_test: 78747 elements
Data y_train: 314987 elements
Data y_test: 78747 elements


In [23]:
#Fase Preprocesamiento cuml
import time
from cuml.preprocessing.LabelEncoder import LabelEncoder

inicio_Pre = time.time()

le = LabelEncoder()

#Convertir a variables categóricas
#seleccionar columnas potenciales para train de xTrain
value_usernameX = le.fit_transform(X_train.username)
value_EventSourceX = le.fit_transform(X_train.event_source)
value_DayX = le.fit_transform(X_train.day)

#seleccionar columnas potenciales para test de xTest
value_usernameXT = le.fit_transform(X_test.username)
value_EventSourceXT = le.fit_transform(X_test.event_source)
value_DayXT = le.fit_transform(X_test.day)

#seleccionar columnas potenciales para train de yTrain
value_usernameY = le.fit_transform(y_train.username)
value_EventSourceY = le.fit_transform(y_train.event_source)
value_DayY = le.fit_transform(y_train.day)

#seleccionar dos columnas potenciales para test de yTest
value_usernameYT = le.fit_transform(y_test.username)
value_EventSourceYT = le.fit_transform(y_test.event_source)
value_DayYT = le.fit_transform(y_test.day)

fin_Pre = time.time()
calculo_Pre_ML = fin_Pre - inicio_Pre

print("Cálculo del proceso de transformación a categóricos con cuml gpu-Rapdis = {}".format(calculo_Pre_ML))

Cálculo del proceso de transformación a categóricos con cuml gpu-Rapdis = 0.18507957458496094


In [31]:
#Fase Uso de los algoritmos ML de cuml: Algoritmo LinearRegression
import time
import cupy as cp
from cuml.linear_model import LinearRegression

inicioRLi = time.time()

#Creación de un nuevo dataframe Train 
dataTrain = cudf.DataFrame()
dataTrain['username'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrain['day'] = cp.asarray(value_DayX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataTest = cudf.Series(new_value)

#Entrenando el algoritmo
lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")
reg = lr.fit(dataTrain, dataTest)

finRLi = time.time()
calculo_Algorit_ML = finRLi - inicioRLi

print("Algoritmo Regresión Lineal")
print("Coeficientes:")
print(reg.coef_)
print('')
print("Intercepción:")
print(reg.intercept_)
print('')
print('Predicción del modelo:')
print(lr.predict(dataTrain))

print("Cálculo del modelo de entrenamiento utilizando el algoritmo Regresion Lineal de cuml = {}".format(calculo_Algorit_ML))

Algoritmo Regresión Lineal
Coeficientes:
0    0.999989
1   -0.000027
dtype: float32

Intercepción:
0.003082275390625

Predicción del modelo:
0         147.000671
1         425.997925
2         509.997345
3         122.001602
4         221.000092
             ...    
314982    221.000092
314983    221.000092
314984    509.997162
314985    221.000092
314986    221.000092
Length: 314987, dtype: float32
Cálculo del modelo de entrenamiento utilizando el algoritmo Regresion Lineal de cuml = 0.010634183883666992


In [39]:
#Fase Uso de los algoritmos ML de cuml: Algoritmo Logistic Regression
import time
import cupy as cp
from cuml.linear_model import LogisticRegression

inicioRL = time.time()

#Creación de un nuevo dataframe Train 
dataTrain = cudf.DataFrame()
dataTrain['username'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrain['day'] = cp.asarray(value_DayX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataTest = cudf.Series(new_value)

#Entrenando el algoritmo
reg_lo = LogisticRegression(fit_intercept = True,
                            max_iter = 1000)
reg_lo.fit(dataTrain, dataTest)

finRL = time.time()
calculo_RL = finRL - inicioRL

print("Coefficients:")
print(reg_lo.coef_)
print("Intercept:")
print(reg_lo.intercept_)
print('Predicción del modelo:')
print(reg_lo.predict(dataTrain))

print('')

print("Cálculo del algoritmo Regresión Logistica con cuml gpu-Rapdis = {}".format(calculo_RL))

[W] [18:45:50.367701] L-BFGS: max iterations reached
Coefficients:
        0         1         2         3         4         5         6    \
0  0.849497  0.928113  0.816582  0.164878  0.933679  0.935568  0.935389   
1  1.807673  0.996173  1.924651  2.423451  1.043672  1.041659  1.009401   

        7         8         9    ...       501       502       503       504  \
0  0.661623  0.667953  0.961438  ...  1.002391  1.042641  1.037800  1.040640   
1  2.256310  2.269264  1.043454  ...  0.952276  0.579827  0.563332  0.603617   

        505       506       507       508       509       510  
0  1.012357  1.021041  1.042625  1.040874  1.037215  1.042989  
1  0.904338  0.841989  0.719991  0.472048  0.712034  0.580196  

[2 rows x 511 columns]
Intercept:
0      1.050005
1      0.998948
2      1.092674
3      1.191396
4      0.999469
         ...   
506    0.981040
507    0.663200
508    0.934540
509    0.899929
510    0.848579
Length: 511, dtype: float32
Predicción del modelo:
0         22